In [ ]:
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from keras import utils as np_utils
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import cv2
import matplotlib.pyplot as plt       
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image         
from tqdm import tqdm
from keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import ImageFile
from collections import namedtuple
import os
      
ImageFile.LOAD_TRUNCATED_IMAGES = True         

# Project Flow

* 0: Import Datasets
* 1: Detect Humans
* 2: Detect Dogs
* 3: Create a CNN to Classify Dog Breeds (from Scratch)
* 4: Use a CNN to Classify Dog Breeds (using Transfer Learning)
* 5: Create a CNN to Classify Dog Breeds (using Transfer Learning)
* 6: Write your Algorithm
* 7: Test Your Algorithm
* 8: Build/deploy webapp

In [ ]:
def load_dog_data(
        data_dir='./dogImages',
        ):
    """ Wrapper for function that loads the dog dataset files.

    Args:
        data_dir (str, optional): Directory to load from. Defaults to './dogImages'.
    """
    def load_dataset(path):
        data = load_files(path)
        dog_files = np.array(data['filenames'])
        dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
        return dog_files, dog_targets

    # load train, test, and validation datasets
    train_files, train_targets = load_dataset('dogImages/train')
    valid_files, valid_targets = load_dataset('dogImages/valid')
    test_files, test_targets = load_dataset('dogImages/test')

    # load list of dog names
    dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

    return (
        train_files, train_targets, 
        valid_files, valid_targets, 
        test_files, test_targets, 
        dog_names
    )


In [ ]:
(train_files, train_targets, 
        valid_files, valid_targets, 
        test_files, test_targets, 
        dog_names
) = load_dog_data()

In [ ]:
# thank you:
# https://www.analyticsvidhya.com/blog/2019/01/build-image-classification-model-10-minutes/?utm_source=blog&utm_source=learn-image-classification-cnn-convolutional-neural-networks-5-datasets
# https://www.analyticsvidhya.com/blog/2018/12/guide-convolutional-neural-network-cnn/

class JustinDogCNN(BaseEstimator, TransformerMixin):
    def __init__(
            self, 
            loss_function='sparse_categorical_crossentropy', 
            optimizer='adam',
            eval_metrics=['accuracy']
            ):
        self.model = None
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.eval_metrics = eval_metrics

    def fit(self, X, y=None):
        input_shape = X.shape[1:]
        self.model = models.Sequential()
        self.model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        self.model.add(layers.MaxPooling2D((2, 2)))
        self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(64, activation='relu'))
        self.model.add(layers.Dense(10, activation='softmax'))  # Assuming 10 classes

        self.model.compile(
            optimizer=self.optimizer,
            loss=self.loss_function,
            metrics=self.eval_metrics,
            )

        self.model.fit(X, y, epochs=10, batch_size=32, verbose=1)
        return self

    def transform(self, X, y=None):
        return self.model.predict(X)
    
pipeline = Pipeline([
    ('cnn', JustinDogCNN())
])

# Fit the pipeline
# pipeline.fit(X_train, y_train)

# Evaluate the model
# test_loss, test_acc = pipeline.named_steps['cnn'].model.evaluate(X_test, y_test, verbose=2)
# print(f"Test accuracy: {test_acc}")